In [1]:
import pandas as pd

path = "/home/carlo/bristoluni/Applied Data Science/git-master/Data/"
filename = path + "water_quality.csv"

In [2]:
df = pd.read_csv(filename)
df = df[['determinand.label', 'result', 'determinand.unit.label', 'water body']]

grouped = df.groupby(['determinand.label', 'determinand.unit.label'])['result'].agg([pd.np.min, pd.np.max, pd.np.mean, pd.np.std, 'count'])
filtered = grouped[grouped['count'] > 15]
#del filtered['count']

In [3]:
filtered

,,amin,amax,mean,std,count
determinand.label,determinand.unit.label,,,,,
Alky pH 4.5,mg/l,171.00000,326.0000,234.973684,32.325613,38
Ammonia(N),mg/l,0.03000,9.1700,0.577450,1.457473,60
B-[a]-pyrene,ug/l,0.00017,0.0229,0.002287,0.005085,20
B-[b]-fluora,ug/l,0.00037,0.0241,0.002732,0.005269,20
B-[ghi]-pery,ug/l,0.00036,0.0313,0.003066,0.006888,20
B-[k]-fluora,ug/l,0.00011,0.0111,0.001155,0.002443,20
BOD ATU,mg/l,8.00000,464.0000,92.280000,164.836414,25
COD as O2,mg/l,52.00000,1255.0000,394.625000,496.830068,16
Cond @ 25C,us/cm,538.00000,2557.0000,903.552632,403.940302,38
